In [2]:
pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [131]:
import pandas as pd


In [132]:
import numpy as np

In [133]:
from sklearn.feature_extraction.text import CountVectorizer

In [134]:
from sklearn.metrics.pairwise import cosine_similarity
from ipywidgets import *

In [135]:
movies=pd.read_csv("movies.csv")
ratings=pd.read_csv("ratings.csv")

In [136]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [137]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [138]:
movies.info

<bound method DataFrame.info of        movieId                                title  \
0            1                     Toy Story (1995)   
1            2                       Jumanji (1995)   
2            3              Grumpier Old Men (1995)   
3            4             Waiting to Exhale (1995)   
4            5   Father of the Bride Part II (1995)   
...        ...                                  ...   
10324   146684        Cosmic Scrat-tastrophe (2015)   
10325   146878           Le Grand Restaurant (1966)   
10326   148238       A Very Murray Christmas (2015)   
10327   148626                 The Big Short (2015)   
10328   149532  Marco Polo: One Hundred Eyes (2015)   

                                            genres  
0      Adventure|Animation|Children|Comedy|Fantasy  
1                       Adventure|Children|Fantasy  
2                                   Comedy|Romance  
3                             Comedy|Drama|Romance  
4                                         

In [139]:
ratings.describe()

,userId,movieId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [140]:
movies.shape

(10329, 3)

In [141]:
ratings.info

<bound method DataFrame.info of         userId  movieId  rating   timestamp
0            1       16     4.0  1217897793
1            1       24     1.5  1217895807
2            1       32     4.0  1217896246
3            1       47     4.0  1217896556
4            1       50     4.0  1217896523
...        ...      ...     ...         ...
105334     668   142488     4.0  1451535844
105335     668   142507     3.5  1451535889
105336     668   143385     4.0  1446388585
105337     668   144976     2.5  1448656898
105338     668   148626     4.5  1451148148

[105339 rows x 4 columns]>

In [142]:
ratings.shape

(105339, 4)

In [143]:
movies['genres']=movies['genres'].str.split("|")

In [144]:
movies2=movies.explode('genres')
movies2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy


In [145]:
movies2['genres'].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'War', 'Musical', 'Documentary',
       'Western', 'Film-Noir', '(no genres listed)'], dtype=object)

In [146]:
movies2['genres'].nunique()

20

In [147]:
movies2=movies2[movies2['genres']!='(no genres listed)']

In [148]:
movies2['genres'].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'War', 'Musical', 'Documentary',
       'Western', 'Film-Noir'], dtype=object)

In [149]:
movies2['genres'].nunique()

19

In [152]:
merged_data=pd.merge(ratings,movies2,on=['movieId'],how='inner')
merged_data

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime
1,1,16,4.0,1217897793,Casino (1995),Drama
2,9,16,4.0,842686699,Casino (1995),Crime
3,9,16,4.0,842686699,Casino (1995),Drama
4,12,16,1.5,1144396284,Casino (1995),Crime
...,...,...,...,...,...,...
281892,668,140098,2.5,1450415424,Runoff (2015),Drama
281893,668,140816,2.5,1443288791,Tangerine (2015),Comedy
281894,668,140816,2.5,1443288791,Tangerine (2015),Drama
281895,668,142488,4.0,1451535844,Spotlight (2015),Thriller


In [153]:
merged_data.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()

genres                                       title    rating     
                                                                mean size
0       Action                                  '71 (2014)  3.500000    1
1       Action     'Hellboy': The Seeds of Creation (2004)  3.000000    1
2       Action                       10 to Midnight (1983)  2.500000    1
3       Action                            12 Rounds (2009)  2.875000    4
4       Action  13 Assassins (Jûsan-nin no shikaku) (2010)  3.500000    5
...        ...                                         ...       ...  ...
23093  Western                           Wyatt Earp (1994)  3.200000   30
23094  Western                           Young Guns (1988)  3.375000   36
23095  Western                        Young Guns II (1990)  3.083333   12
23096  Western                           Young Ones (2014)  2.000000    1
23097  Western                       ¡Three Amigos! (1986)  3.012500   40

[23098 rows x 4 columns]

In [154]:
popularity=merged_data.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()
popularity.columns=["Genres","Title","Average Ratings","Number of Ratings"]
popularity

,Genres,Title,Average Ratings,Number of Ratings
0,Action,'71 (2014),3.500000,1
1,Action,'Hellboy': The Seeds of Creation (2004),3.000000,1
2,Action,10 to Midnight (1983),2.500000,1
3,Action,12 Rounds (2009),2.875000,4
4,Action,13 Assassins (Jûsan-nin no shikaku) (2010),3.500000,5
...,...,...,...,...
23093,Western,Wyatt Earp (1994),3.200000,30
23094,Western,Young Guns (1988),3.375000,36
23095,Western,Young Guns II (1990),3.083333,12
23096,Western,Young Ones (2014),2.000000,1


In [155]:
#genre+actiom
#threshold=50

popularity[(popularity["Genres"]=='Action')&(popularity['Number of Ratings']>=50)]

,Genres,Title,Average Ratings,Number of Ratings
13,Action,28 Days Later (2002),3.932836,67
20,Action,300 (2007),3.577922,77
43,Action,"Abyss, The (1989)",3.681319,91
62,Action,Air Force One (1997),3.439024,82
71,Action,Aliens (1986),4.146497,157
...,...,...,...,...
1638,Action,V for Vendetta (2006),4.015957,94
1676,Action,Waterworld (1995),2.934959,123
1692,Action,Wild Wild West (1999),2.164179,67
1707,Action,X-Men (2000),3.557692,130


In [156]:
#genre+actiom
#threshold=50
#top=7
popularity[(popularity["Genres"]=='Action')&(popularity['Number of Ratings']>=50)].sort_values(by='Average Ratings',ascending=False).head(7)

,Genres,Title,Average Ratings,Number of Ratings
1179,Action,Princess Mononoke (Mononoke-hime) (1997),4.384615,52
1076,Action,North by Northwest (1959),4.273973,73
975,Action,"Matrix, The (1999)",4.264368,261
1433,Action,Star Wars: Episode V - The Empire Strikes Back...,4.228070,228
1331,Action,Seven Samurai (Shichinin no samurai) (1954),4.217742,62
1199,Action,Raiders of the Lost Ark (Indiana Jones and the...,4.212054,224
747,Action,Inception (2010),4.189320,103


In [192]:
def TopNPopularMovies(genre,threshold,topN):
    popularity=merged_data.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()
    popularity.columns=["Genres","Title","Average Ratings","Number of Ratings"]

    topNrecommendations=popularity[(popularity['Genres']==genre)&(popularity['Number of Ratings']>=threshold)].sort_values(by='Average Ratings',ascending=False).head(topN)

#outputb
    topNrecommendations['Sno.']=range(1,len(topNrecommendations)+1)
    topNrecommendations.index=range(0,len(topNrecommendations)+1)
    topNrecommendations.columns=['Genres','Movie Title','Average Movie Rating','Number of Reviews','Sno.']
    return topNrecommendations[['Sno.','Movie Title','Average Movie Rating','Number of Reviews']]

In [204]:
def TopNPopularMovies(genre, threshold, topN):
    popularity = merged_data.groupby(['genres', 'title']).agg({'rating': ['mean', 'size']}).reset_index()
    popularity.columns = ["Genres", "Title", "Average Ratings", "Number of Ratings"]

    topNrecommendations = popularity[(popularity['Genres'] == genre) & (popularity['Number of Ratings'] >= threshold)].sort_values(by='Average Ratings', ascending=False).head(topN)

    # Output
    topNrecommendations['Sno.'] = range(1, len(topNrecommendations) + 1)
    
    # topNrecommendations.index = range(0, len(topNrecommendations) + 1)
    topNrecommendations.columns = ['Genres', 'Movie Title', 'Average Movie Rating', 'Number of Reviews', 'Sno.']
    return topNrecommendations[['Sno.', 'Movie Title', 'Average Movie Rating', 'Number of Reviews']]


In [205]:
#part4
genre='adventure'
threshold=50
topN=8
TopNPopularMovies(genre=genre,threshold=threshold,topN=topN)

,Sno.,Movie Title,Average Movie Rating,Number of Reviews


In [159]:
movies2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy


In [206]:
#movies3=pd.read_csv("movies.csv")

In [214]:
movies3=movies2.groupby('title').agg({"genres":lambda x:" ".join(list(x))})

In [215]:
movies3.head()

,genres
title,
'71 (2014),Action Drama Thriller War
'Hellboy': The Seeds of Creation (2004),Action Adventure Comedy Documentary Fantasy
'Round Midnight (1986),Drama Musical
'Til There Was You (1997),Drama Romance
"'burbs, The (1989)",Comedy


In [216]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [217]:
tf=TfidfVectorizer(analyzer='word',ngram_range=(1,3),stop_words='english')
tf

TfidfVectorizer(ngram_range=(1, 3), stop_words='english')

In [218]:
tf_matrix=tf.fit_transform(movies3['genres'])

In [219]:
cosine_sim=cosine_similarity(tf_matrix,tf_matrix)

In [211]:
cosine_sim

array([[1.        , 0.02677945, 0.02931913, ..., 0.10229517, 0.        ,
        0.        ],
       [0.02677945, 1.        , 0.        , ..., 0.03626651, 0.02411583,
        0.02863994],
       [0.02931913, 0.        , 1.        , ..., 0.        , 0.        ,
        0.35526663],
       ...,
       [0.10229517, 0.03626651, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.02411583, 0.        , ..., 0.        , 1.        ,
        0.07090711],
       [0.        , 0.02863994, 0.35526663, ..., 0.        , 0.07090711,
        1.        ]])

In [220]:
movies3.head()

,genres
title,
'71 (2014),Action Drama Thriller War
'Hellboy': The Seeds of Creation (2004),Action Adventure Comedy Documentary Fantasy
'Round Midnight (1986),Drama Musical
'Til There Was You (1997),Drama Romance
"'burbs, The (1989)",Comedy


In [223]:
movies3=pd.read_csv("movies.csv")

In [224]:
movies3['title']

0                           Toy Story (1995)
1                             Jumanji (1995)
2                    Grumpier Old Men (1995)
3                   Waiting to Exhale (1995)
4         Father of the Bride Part II (1995)
                        ...                 
10324          Cosmic Scrat-tastrophe (2015)
10325             Le Grand Restaurant (1966)
10326         A Very Murray Christmas (2015)
10327                   The Big Short (2015)
10328    Marco Polo: One Hundred Eyes (2015)
Name: title, Length: 10329, dtype: object

In [225]:
pd.Series(movies3.index,index=movies3['title'])

title
Toy Story (1995)                           0
Jumanji (1995)                             1
Grumpier Old Men (1995)                    2
Waiting to Exhale (1995)                   3
Father of the Bride Part II (1995)         4
                                       ...  
Cosmic Scrat-tastrophe (2015)          10324
Le Grand Restaurant (1966)             10325
A Very Murray Christmas (2015)         10326
The Big Short (2015)                   10327
Marco Polo: One Hundred Eyes (2015)    10328
Length: 10329, dtype: int64

In [179]:
def recommendation_genre(movie_df, similarity_matrix, movie_title, topN):
    # Indices for all movies
    indices = pd.Series(movie_df.index, index=movie_df['title'])
    
    # Index of the target movie
    index = indices[movie_title]
    
    # Cosine similarity
    cosine_scores = list(enumerate(similarity_matrix[index]))
    cosine_scores = sorted(cosine_scores, key=lambda x: x[1], reverse=True)[1:topN + 2]
    
    # Extract the matching movies
    matched = [i[0] for i in cosine_scores]
    
    # Extract matching dataframe
    matching_df = movie_df.iloc[matched]
    
    # Filter out the target movie
    matching_df = matching_df[matching_df['title'] != movie_title]

    # Output
    matching_df.rename(columns={'title': 'Movie Title'}, inplace=True)
    matching_df['Sno.'] = range(1, len(matching_df) + 1)
    matching_df.index = range(0, len(matching_df))
    return matching_df[['Sno.', 'Movie Title']].head(topN)


In [180]:
#testcase
recommendation_genre(movie_df=movies3, similarity_matrix=cosine_sim, movie_title='Contraband (2012)', topN=10)

,Sno.,Movie Title
0,1,Boys (1996)
1,2,Jude (1996)
2,3,Funeral in Berlin (1966)
3,4,Scanners (1981)
4,5,Mogambo (1953)
5,6,"Mean Season, The (1985)"
6,7,Angus (1995)
7,8,All About Eve (1950)
8,9,Cinderella (1950)
9,10,Manon of the Spring (Manon des sources) (1986)


In [181]:
#colaboratory filtering system

In [182]:
#popularity#inputs
genres=Dropdown(options=list(set(movies2['genres'])),description='Genres',style={"descriptive_width":'initial'})
num_reviews=IntText(description="Minimum Reviews",style={"descriptive_width":'initial'})
num_recomendations_1=IntText(description="Number of Recommendation",style={"descriptive_width":'initial'})

#tabs
b1=Button(description="RECOMMEND ME",style={"descriptive_width":'initial'})
h1=HBox([num_reviews,num_recomendations_1])
popularity_tab=VBox([genres,h1,b1])

#content bases system
title=Textarea(description="Movie Title",style={"descriptive_width":'initial'})
num_recomendations_2=IntText(description="Number of Recommendation",style={"descriptive_width":'initial'})

#tabs
h2=HBox([title,num_recomendations_2])
b2=Button(description="RECOMMEND ME",style={"descriptive_width":'initial'})

content_tab=VBox([h2,b2])

#creating final tab
tabs=[popularity_tab,content_tab]
wid=widgets.Tab(tabs)

#set titles to the tabs
names=['Popularity Based Recommendations','Content Based Recommendations']
[wid.set_title(i,title) for i,title in enumerate(names)]

display(wid)

In [188]:
#setting up events to respond 
#popularity
def b1_clicked(b):
    global output
    output=TopNPopularMovies(genre=genres.value,threshold=num_reviews.value,topN=num_recomendations_1.value)
b1.on_click(b1_clicked)  

#content 
def b2_clicked(b):
    global output
    result=recommendation_genre(movie_df=movies3, similarity_matrix=cosine_sim, movie_title=title.value, topN=num_recomendations_2.value)
    #output=result
b2.on_click(b2_clicked)  
 

In [184]:
display(wid)

In [235]:
output


,Sno.,Movie Title,Average Movie Rating,Number of Reviews
18641,1,North by Northwest (1959),4.273973,73
18395,2,Life Is Beautiful (La Vita è bella) (1997),4.253425,73
17799,3,Casablanca (1942),4.236000,125
18132,4,Good Will Hunting (1997),4.167857,140
18774,5,"Princess Bride, The (1987)",4.163743,171
17606,6,American Beauty (1999),4.157407,216
18068,7,Forrest Gump (1994),4.138264,311
18140,8,"Graduate, The (1967)",4.132184,87
18000,9,Eternal Sunshine of the Spotless Mind (2004),4.122449,98
17604,10,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",4.075630,119
